In [1]:
import pandas as pd
import glob
from tqdm import tqdm
import requests
import time
import os
import shutil
import matplotlib.pyplot as plt
import sys
import platform

def load_manifest_file(location):
    """
    Function use:
    Load the GDC manifest file from a specified directory.

    Inputs:
    - location (str): Directory path containing the GDC manifest file.

    Outputs:
    - DataFrame: Contains the data from the manifest file if found.
    - None: If no matching file is found.
    """
    pattern = f"{location}/*gdc_manifest*"
    files = glob.glob(pattern)
    if files:
        return pd.read_csv(files[0], sep='\t')
    else:
        return None

def fetch_sample_type_from_file_uuid(file_uuid):
    """
    Function use:
    Fetch the sample type for a given file UUID using the GDC API.

    Inputs:
    - file_uuid (str): UUID of the file.

    Outputs:
    - sample_type (str): Sample type (e.g., 'Primary Tumor').
    - None: If the API request fails.
    """
    base_url = 'https://api.gdc.cancer.gov'
    response = requests.get(f'{base_url}/files/{file_uuid}?expand=cases.samples')
    if response.status_code == 200:
        data = response.json()
        return data.get('data', {}).get('cases', [{}])[0].get('samples', [{}])[0].get('sample_type', 'Not Found')
    else:
        print(f"Failed to retrieve sample type for UUID {file_uuid}.")
        return None

def fetch_diagnosis_details_from_file_uuid(file_uuid):
    """
    Function use:
    Fetch diagnosis details for a given file UUID using the GDC API.

    Inputs:
    - file_uuid (str): UUID of the file.

    Outputs:
    - diagnosis_details (list): List of dictionaries containing diagnosis information.
    - Empty list: If the API request fails.
    """
    base_url = 'https://api.gdc.cancer.gov'
    response = requests.get(f'{base_url}/files/{file_uuid}?expand=cases.diagnoses')
    if response.status_code == 200:
        data = response.json()
        return data.get('data', {}).get('cases', [{}])[0].get('diagnoses', [])
    else:
        print(f"Failed to retrieve diagnosis details for UUID {file_uuid}.")
        return []

def fetch_clinical_data_for_manifest(manifest, n=None, t=1):
    """
    Function use:
    Enrich a manifest DataFrame with sample types and diagnosis details fetched from the GDC API.

    Inputs:
    - manifest (DataFrame): Manifest with file UUIDs in an 'id' column.
    - n (int): Maximum number of files to process. If None, all rows are processed.
    - t (float): Time (in seconds) to wait between API calls.

    Outputs:
    - Updated DataFrame: Includes additional sample type and diagnosis details.
    """
    if 'sample_type' not in manifest.columns:
        manifest['sample_type'] = None

    if n is None:
        n = len(manifest)

    for i in tqdm(range(min(n, len(manifest)))):
        if i > 0:
            time.sleep(t)
        file_id = manifest.loc[i, 'id']
        sample_type = fetch_sample_type_from_file_uuid(file_id)
        if sample_type:
            manifest.at[i, 'sample_type'] = sample_type
        else:
            print(f"No sample type found for file UUID {file_id}.")
        diagnosis_details = fetch_diagnosis_details_from_file_uuid(file_id)
        if diagnosis_details and isinstance(diagnosis_details, list):
            for detail in diagnosis_details:
                if isinstance(detail, dict):
                    for key, value in detail.items():
                        column_name = f"diagnosis_{key}"
                        if column_name not in manifest.columns:
                            manifest[column_name] = None
                        manifest.at[i, column_name] = value
                else:
                    print(f"Detail is not a dictionary: {detail}")
        else:
            print(f"Diagnosis details structure unexpected or missing for file ID {file_id}: {diagnosis_details}")

    return manifest

def save_clinical_manifest(manifest, directory):
    """
    Function use:
    Save the clinical manifest DataFrame to a file.

    Inputs:
    - manifest (DataFrame): Data to save.
    - directory (str): Directory path where the file will be saved.

    Outputs:
    - None: File is saved directly to the specified directory.
    """
    directory = os.path.join(directory, '')
    filepath = f"{directory}clinical_gdc_manifest.txt"
    manifest.to_csv(filepath, sep='\t', index=False)
    print(f"File saved successfully at {filepath}")

def load_clinical_manifest(directory):
    """
    Function use:
    Load a clinical manifest file into a DataFrame.

    Inputs:
    - directory (str): Directory path where the file is located.

    Outputs:
    - DataFrame: Contains the clinical manifest data.
    """
    filepath = os.path.join(directory, 'clinical_gdc_manifest.txt')
    clinical_manifest = pd.read_csv(filepath, sep='\t')
    print('Clinical manifest shape:', clinical_manifest.shape)
    return clinical_manifest

def clean_dataframe(df):
    """
    Function use:
    Clean a DataFrame by removing columns with irrelevant or missing data.

    Inputs:
    - df (DataFrame): DataFrame to clean.

    Outputs:
    - DataFrame: Cleaned version of the input DataFrame.
    """
    df_copy = df.copy()
    for column in df_copy.columns:
        unique_values = df_copy[column].dropna().unique()
        if len(unique_values) == 0 or (len(unique_values) == 1 and unique_values[0] in ['Not Reported', 'not reported', 'None']):
            df_copy.drop(column, axis=1, inplace=True)
        else:
            if column in ["ajcc_pathologic_stage", "tumor_grade", "ajcc_pathologic_t", "ajcc_pathologic_m",
                          "ajcc_pathologic_n", "figo_stage", "wilms_tumor_histologic_subtype"]:
                df_copy.loc[df_copy['sample_type'] == 'Solid Tissue Normal', column] = 'healthy tissue'
    print('Cleaned clinical manifest shape:', df_copy.shape)
    return df_copy

def organize_tsv_files(base_dir):
    """
    Function use:
    Organize .tsv files by moving them from subdirectories to the base directory.

    Inputs:
    - base_dir (str): Path to the base directory.

    Outputs:
    - None: Files are moved, and subdirectories are cleaned.
    """
    has_directories = False
    for root, dirs, files in os.walk(base_dir, topdown=False):
        if dirs:
            has_directories = True
        for file in files:
            if file.endswith(".tsv"):
                shutil.move(os.path.join(root, file), os.path.join(base_dir, file))
    if has_directories:
        for root, dirs, files in os.walk(base_dir, topdown=False):
            for dir in dirs:
                shutil.rmtree(os.path.join(root, dir))
        print("Cleaning done")
    else:
        print("No directories to clean. Cleaning done.")

def load_gene_expression(manifest_df, exp_dir, feature):
    """
    Function use:
    Load gene expression data from .tsv files, keeping only relevant rows and columns.

    Inputs:
    - manifest_df (DataFrame): Manifest with filenames.
    - exp_dir (str): Directory containing .tsv files.
    - feature (str): Gene expression feature to extract.

    Outputs:
    - gene_expression_df (DataFrame): Contains gene expression data.
    - gene_key_df (DataFrame): Maps gene IDs to gene names.
    """
    gene_expression_data = {}
    gene_key = {}

    for filename in manifest_df['filename']:
        file_path = os.path.join(exp_dir, filename)
        try:
            df = pd.read_csv(file_path, sep='\t', comment='#')
            df = df[df['gene_id'].str.contains('ENSG')]
            gene_expression_data[filename] = df[feature].values
            gene_key.update(dict(zip(df['gene_id'], df['gene_name'])))
        except Exception as e:
            print(f"Failed to load {filename}: {e}")

    gene_expression_df = pd.DataFrame(gene_expression_data, index=list(gene_key.keys()))
    gene_expression_df.index.name = 'gene_id'
    gene_expression_df.columns.name = 'filename'

    gene_key_df = pd.DataFrame(list(gene_key.items()), columns=['gene_id', 'gene_name'])

    print('Gene expression shape:', gene_expression_df.T.shape)
    print('Gene key shape:', gene_key_df.shape)

    return gene_expression_df.T, gene_key_df

def save_dataframes(gene_exps, gene_key, cancer):
    """
    Function use:
    Save gene expression and key DataFrames to files.

    Inputs:
    - gene_exps (DataFrame): Gene expression data.
    - gene_key (DataFrame): Gene key data.
    - cancer (str): Directory for saving the data.

    Outputs:
    - None: DataFrames are saved to files.
    """
    gene_exps.to_csv(f'./data/TCGA_GeneExpression/{cancer}/gene_expression.csv', index=True)
    gene_key.to_csv(f'./data/TCGA_GeneExpression/{cancer}/gene_key.csv', index=True)

def load_dataframes(exps_filename, key_filename):
    """
    Function use:
    Load gene expression and key DataFrames from files.

    Inputs:
    - exps_filename (str): Path to gene expression file.
    - key_filename (str): Path to gene key file.

    Outputs:
    - gene_exps (DataFrame): Loaded gene expression data.
    - gene_key (DataFrame): Loaded gene key data.
    """
    gene_exps = pd.read_csv(exps_filename, index_col=0)
    gene_key = pd.read_csv(key_filename, index_col=0)
    return gene_exps, gene_key

def analyze_filenames(manifest_df, directory_path):
    """
    Analyze the uniqueness and overlap of filenames between a manifest and a directory.

    Inputs:
    - manifest_df (DataFrame): Manifest containing filenames.
    - directory_path (str): Directory path containing files to compare.

    Outputs:
    - dict: Summary of percentages including unique filenames, overlap, and non-overlap.
    """
    total_filenames = len(manifest_df['filename'])
    unique_filenames = len(manifest_df['filename'].unique())
    percent_unique = (unique_filenames / total_filenames) * 100

    directory_files = set(os.listdir(directory_path))
    manifest_files = set(manifest_df['filename'])

    overlap_files = manifest_files.intersection(directory_files)
    percent_overlap = (len(overlap_files) / total_filenames) * 100
    non_overlap_files = manifest_files.difference(directory_files)
    percent_non_overlap = (len(non_overlap_files) / total_filenames) * 100

    return {
        "percent_unique_filenames_in_manifest": percent_unique,
        "percent_overlap_with_directory": percent_overlap,
        "percent_non_overlap_with_directory": percent_non_overlap
    }

# Show Python version, platform details, and versions of all imported packages.
def environment_info():
    """
    Display Python version, platform details, and versions of all imported packages.
    """
    # Python and system information
    print("Python version:", sys.version)
    print("Platform:", platform.platform())

    # Imported package versions
    print("\nPackage Versions:")
    imported_modules = {name: module.__version__ for name, module in sys.modules.items() 
                        if hasattr(module, '__version__')}
    for package, version in sorted(imported_modules.items()):
        print(f"{package}: {version}")


In [2]:
# Display environment detail
environment_info()


Python version: 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 21:00:12) [Clang 16.0.6 ]
Platform: macOS-14.5-x86_64-i386-64bit

Package Versions:
IPython: 8.25.0
IPython.core.release: 8.25.0
PIL: 10.3.0
PIL.Image: 10.3.0
PIL._deprecate: 10.3.0
PIL._version: 10.3.0
_brotli: 1.0.9
_csv: 1.0
_ctypes: 1.1.0
_curses: b'2.2'
_decimal: 1.70
_pydev_bundle.fsnotify: 0.1.5
_pydevd_frame_eval.vendored.bytecode: 0.13.0.dev
appnope: 0.1.4
argparse: 1.1
brotli: 1.0.9
certifi: 2024.02.02
cffi: 1.16.0
charset_normalizer: 2.0.4
charset_normalizer.version: 2.0.4
comm: 0.2.2
csv: 1.0
ctypes: 1.1.0
ctypes.macholib: 1.0
cycler: 0.12.1
dateutil: 2.9.0
dateutil._version: 2.9.0
debugpy: 1.6.7
debugpy.public_api: 1.6.7
decimal: 1.70
decorator: 5.1.1
defusedxml: 0.7.1
executing: 2.0.1
executing.version: 2.0.1
http.server: 0.6
idna: 3.4
idna.idnadata: 15.0.0
idna.package_data: 3.4
ipaddress: 1.0
ipykernel: 6.29.3
ipykernel._version: 6.29.3
jedi: 0.19.1
json: 2.0.9
jupyter_client: 8.6.2
jupyter_client._v

In [2]:
# Specify the cancer type to analyze 
# Analyzed in manuscript ("uterus", "thyroid", "lung", "kidney", "breast", "bone_marrow_and_blood")
cancer = "uterus"

# --- Prerequisites ---
# Before running this script, ensure the following:
# 1. A directory structure exists: './data/TCGA_GeneExpression/{cancer}/'
#    (e.g., './data/TCGA_GeneExpression/uterus/').
# 2. Within the above directory, the following must be present:
#    - A GDC manifest txt file with 'gdc_manifest' in its name. This file must be downloaded
#      from the GDC Data Portal and placed in './data/TCGA_GeneExpression/{cancer}/'.
#    - A subdirectory named 'gene_expression' containing all downloaded gene expression `.tsv` files
#      (also from the GDC Data Portal) for the specified cancer type.  
#      (select cancer type filter > transcriptome profiling > Gene Expression Quantification > RNA-Seq > '.tsv' files)


# --- Step 1: Load the manifest file from GDC ---
# This step loads the manifest file listing metadata about available files for the specified cancer type.
gdc_manifest = load_manifest_file(f'./data/TCGA_GeneExpression/{cancer}/')

# --- Step 2: Fetch clinical data using the GDC API ---
# This step queries the GDC API to fetch clinical data such as diagnosis details and sample types.
# Ensure an active internet connection to access the GDC API.
clinical_manifest = fetch_clinical_data_for_manifest(gdc_manifest, n=None, t=0)

# Save the clinical data for reuse. This file will be saved as 'clinical_gdc_manifest.txt'.
save_clinical_manifest(clinical_manifest, f'./data/TCGA_GeneExpression/{cancer}/')

# --- Step 3: Load and clean the clinical manifest ---
# Load the saved clinical manifest file and clean it by removing irrelevant or empty columns.
clinical_manifest = load_clinical_manifest(f'./data/TCGA_GeneExpression/{cancer}/')
clinical_manifest = clean_dataframe(clinical_manifest)

# --- Step 4: Organize and compile gene expression data ---
# Ensure that the subdirectory './data/TCGA_GeneExpression/{cancer}/gene_expression/' exists
# and contains the downloaded gene expression `.tsv` files from the GDC Data Portal.

# Move all `.tsv` files from subdirectories into the 'gene_expression' directory.
organize_tsv_files(f'./data/TCGA_GeneExpression/{cancer}/gene_expression')

# Load the gene expression data from the `.tsv` files, extracting the specified feature ('fpkm_unstranded').
gene_exps, gene_key = load_gene_expression(
    clinical_manifest, f'./data/TCGA_GeneExpression/{cancer}/gene_expression', 'fpkm_unstranded'
)

# Save the processed gene expression and gene key data for later use.
save_dataframes(gene_exps, gene_key, cancer)

# --- Step 5: Analyze and verify filenames ---
# Compare filenames in the clinical manifest with those in the 'gene_expression' directory
# to identify any mismatches or missing files.
print(analyze_filenames(clinical_manifest, f'./data/TCGA_GeneExpression/{cancer}/gene_expression'))

# --- Step 6: Reload saved gene expression and gene key data ---
# Reload the saved gene expression and gene key data from the CSV files.
# This is useful for reanalysis or downstream processing without repeating earlier steps.
gene_exps, gene_key = load_dataframes(
    f'./data/TCGA_GeneExpression/{cancer}/gene_expression.csv', 
    f'./data/TCGA_GeneExpression/{cancer}/gene_key.csv'
)


100%|███████████████████████████████████████| 1168/1168 [11:45<00:00,  1.66it/s]


File saved successfully at ./data/TCGA_GeneExpression/uterus/clinical_gdc_manifest.txt
Clinical manifest shape: (1168, 102)
Cleaned clinical manifest shape: (1168, 44)
No directories to clean. Cleaning done.
Gene expression shape: (1168, 60660)
Gene key shape: (60660, 2)
{'percent_unique_filenames_in_manifest': 100.0, 'percent_overlap_with_directory': 100.0, 'percent_non_overlap_with_directory': 0.0}


In [3]:
gene_key.head()

,gene_id,gene_name
0,ENSG00000000003.15,TSPAN6
1,ENSG00000000005.6,TNMD
2,ENSG00000000419.13,DPM1
3,ENSG00000000457.14,SCYL3
4,ENSG00000000460.17,C1orf112


In [4]:
gene_exps.head()

,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
filename,,,,,,,,,,,,,,,,,,,,,
6a6ccdf5-9405-409a-a161-72e875400889.rna_seq.augmented_star_gene_counts.tsv,19.2975,1.4941,12.5086,6.7530,3.9497,1.4546,16.7088,12.2630,20.4519,16.5588,...,0.0,36.7550,0.8335,0.0,44.4953,0.0,8.2613,0.0,0.0176,1.0098
e5325dd8-867a-4cc2-8b93-9b0b58fc32f3.rna_seq.augmented_star_gene_counts.tsv,10.6894,0.3790,35.1229,5.4748,2.5248,3.5777,3.1174,8.7109,4.1676,13.2483,...,0.0,7.3599,0.1395,0.0,11.5829,0.0,5.5882,0.0,0.0673,0.0350
75073929-f7b1-4165-a786-60f733b962d9.rna_seq.augmented_star_gene_counts.tsv,17.7992,0.4563,26.4298,5.0877,2.7658,4.6093,29.4178,15.9268,8.3757,9.7255,...,0.0,6.6512,0.1094,0.0,13.9021,0.0,2.9262,0.0,0.0389,0.3266
119ac668-eab5-42e5-a7a5-f28720173235.rna_seq.augmented_star_gene_counts.tsv,10.4508,0.3030,18.1245,9.1775,5.8430,0.8595,11.2738,14.0382,4.9385,15.3237,...,0.0,9.9381,0.1635,0.0,25.6598,0.0,5.1973,0.0,0.0387,0.2218
09c2ef25-0496-4574-995b-d493f9ddf724.rna_seq.augmented_star_gene_counts.tsv,27.1184,0.3620,26.7232,6.2870,3.4341,3.0606,55.9515,17.7718,20.5075,15.8846,...,0.0,14.4691,0.0762,0.0,20.8004,0.0,3.2337,0.0,0.0347,0.1391
